An alternate way to handle this problem is to train two separate resnet models for the two image tasks. We can then use Tonks to combine them into an ensemble model with the text model that was trained on both tasks.

This notebook trains a gender model, Step6 trains a season model, but they could be run in parallel.

This notebook was run on an AWS p3.2xlarge

# Tonks Image Model Training Pipeline

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

Note: for images, we use the MultiInputMultiTaskLearner since we will send in the full image and a center crop of the image.

In [4]:
from tonks import MultiInputMultiTaskLearner, MultiDatasetLoader
from tonks.vision.dataset import TonksImageDataset
from tonks.vision.models import ResnetForMultiTaskClassification

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [5]:
TRAIN_GENDER_DF = pd.read_csv('/home/ec2-user/fashion_dataset/gender_train.csv')

In [6]:
VALID_GENDER_DF = pd.read_csv('/home/ec2-user/fashion_dataset/gender_valid.csv')

In [7]:
#TRAIN_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_train.csv')

In [8]:
#VALID_SEASON_DF = pd.read_csv('/home/ubuntu/fashion_dataset/season_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [9]:
batch_size = 64

We use the `TonksImageDataSet` class to create train and valid datasets for each task.

Check out the documentation for infomation about the transformations.

In [10]:
gender_train_dataset = TonksImageDataset(
    x=TRAIN_GENDER_DF['image_urls'],
    y=TRAIN_GENDER_DF['gender_cat'],
    transform='train',
    crop_transform='train'
)
gender_valid_dataset = TonksImageDataset(
    x=VALID_GENDER_DF['image_urls'],
    y=VALID_GENDER_DF['gender_cat'],
    transform='val',
    crop_transform='val'
)

# season_train_dataset = TonksImageDataset(
#     x=TRAIN_SEASON_DF['image_urls'],
#     y=TRAIN_SEASON_DF['season_cat'],
#     transform='train',
#     crop_transform='train'
# )
# season_valid_dataset = TonksImageDataset(
#     x=VALID_SEASON_DF['image_urls'],
#     y=VALID_SEASON_DF['season_cat'],
#     transform='val',
#     crop_transform='val'
# )

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [11]:
train_dataloaders_dict = {
    'gender': DataLoader(gender_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    #'season': DataLoader(season_train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
}
valid_dataloaders_dict = {
    'gender': DataLoader(gender_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
    #'season': DataLoader(season_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4),
}

The dictionary of dataloaders is then put into an instance of the Tonks `MultiDatasetLoader` class.

In [12]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

417

In [13]:
ValidLoader = MultiDatasetLoader(loader_dict=valid_dataloaders_dict, shuffle=False)
len(ValidLoader)

139

We need to create a dictionary of the tasks and the number of unique values so that we can create our model.

In [14]:
new_task_dict = {
    'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
    #'season': TRAIN_SEASON_DF['season_cat'].nunique(),
}

In [15]:
new_task_dict

{'gender': 5}

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Create Model and Learner
===

These are completely new tasks so we use `new_task_dict`. If we had already trained a model on some tasks, we would use `pretrained_task_dict`.

And since these are new tasks, we set `load_pretrained_renset=True` to use the weights from Torch.

In [16]:
model = ResnetForMultiTaskClassification(
    new_task_dict=new_task_dict,
    load_pretrained_resnet=True
)

You will likely need to explore different values in this section to find some that work
for your particular model.

In [17]:
lr_last = 1e-2
lr_main = 1e-4

optimizer = optim.Adam([
    {'params': model.resnet.parameters(), 'lr': lr_main},
    {'params': model.dense_layers.parameters(), 'lr': lr_last},
    {'params': model.new_classifiers.parameters(), 'lr': lr_last},
    
])

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [18]:
loss_function_dict = {'gender': 'categorical_cross_entropy', 'season': 'categorical_cross_entropy'}
metric_function_dict = {'gender': 'multi_class_acc', 'season': 'multi_class_acc'}

In [19]:
learn = MultiInputMultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict, loss_function_dict, metric_function_dict)

Train model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [20]:
learn.fit(
    num_epochs=10,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,gender_train_loss,gender_val_loss,gender_multi_class_accuracy,time
0.552644,0.005725,0.552644,0.005725,0.867950,03:00
0.426063,0.005540,0.426063,0.005540,0.874367,02:59
0.384687,0.004857,0.384687,0.004857,0.893730,02:59
0.357850,0.005062,0.357850,0.005062,0.899358,02:59
0.276738,0.004644,0.276738,0.004644,0.906901,02:59
0.251222,0.004448,0.251222,0.004448,0.912755,02:59
0.238900,0.004213,0.238900,0.004213,0.915119,02:59
0.222322,0.004059,0.222322,0.004059,0.918721,02:59
0.212273,0.004060,0.212273,0.004060,0.918271,02:59
0.203351,0.004262,0.203351,0.004262,0.917370,02:59


Epoch 7 best model saved with loss of 0.004059127066284418


Validate model
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [21]:
pred_dict = learn.get_val_preds(device)

In [22]:
pred_dict

{'gender': {'y_true': array([4, 2, 2, ..., 2, 1, 2]),
  'y_pred': array([[1.2556822e-10, 3.5671825e-08, 9.0080803e-06, 3.4118258e-07,
          9.9999058e-01],
         [2.8283958e-04, 2.5466916e-06, 9.9344581e-01, 3.5342309e-03,
          2.7345356e-03],
         [9.6761892e-07, 2.5201075e-09, 9.9999738e-01, 1.2567381e-06,
          3.9190229e-07],
         ...,
         [1.5187657e-06, 1.2129793e-08, 9.9999130e-01, 3.3350207e-06,
          3.8055907e-06],
         [8.4280781e-04, 9.8094398e-01, 2.9174918e-05, 5.7934500e-05,
          1.8126275e-02],
         [2.6078537e-04, 3.7668874e-06, 9.8065370e-01, 1.6031884e-02,
          3.0498395e-03]], dtype=float32)}}

Save/Export Model
===

Once we are happy with our training we can save (or export) our model, using the `save` method (or `export`).

See the docs for the difference between `save` and `export`.

We will need the saved model later to use in the ensemble model

In [23]:
model.save(folder='/home/ec2-user/fashion_dataset/models/', model_id='GENDER_IMAGE_MODEL1')

In [24]:
model.export(folder='/home/ec2-user/fashion_dataset/models/', model_id='GENDER_IMAGE_MODEL1')

Now that we have a gender image model, we can move to `Step6_train_season_image_model`.